### Importing libraries

In [1]:
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
import matplotlib
from pylab import rcParams
from pmdarima.arima import auto_arima

matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'

### Reading data and getting in the right format

In [2]:
def read_stock_data(path):
    df = pd.read_csv(path)
    df = df.sort_values('Date')
    return df

### ARIMA Model

In [3]:
def create_decision_data(df, date, ls):
    
#     df = df[df['Date'] <= date]
    
    dl = [['Stock Symbol','last_day_value','next_day_forecast','AIC','next_day_value']]

    for i in ls: #### Change later
        try:
            df2 = df.loc[df['Stock Symbol'] == i].drop(columns = 'Stock Symbol')
            df2.isnull().sum()
            df2['Date'] = pd.to_datetime(df2['Date'], format='%Y-%m-%d', errors='coerce')
            df2t = df2[df2['Date'] <= date]
            df2v = df2[df2['Date'] > date]
            df2t = df2t.set_index('Date')
            df2v = df2v.set_index('Date')

            model = auto_arima(df2t, trace=True, error_action='ignore', suppress_warnings=True)
            model.fit(df2t)

            next_day_forecast = model.predict(n_periods=1)[0]
            AIC = model.aic()
            next_day_value = df2v.iloc[:1,].values[0][0]
            last_day_value = df2t.iloc[-1:,].values[0][0]
            dl.append([i, last_day_value, next_day_forecast, AIC, next_day_value])
        except:
            print('Error with ',i)

    df3 = pd.DataFrame(dl)
    df3.columns = df3.iloc[0]
    df3 = df3.drop(df3.index[0])
    df3['diff'] = df3['next_day_forecast'] - df3['last_day_value']
    df3 = df3.sort_values('diff', ascending = False)
    return df3

### Update Ledger for available funds

In [4]:
def update_ledger(df3, path):
    try:
        dfl = pd.read_csv(path)
        dfl2 = dfl.merge(df3, on = 'Stock Symbol')
        dfl2['Investment'] = dfl2['Investment']*(dfl2['last_day_value']/dfl2['Bought at'])
        dfl2['Bought at'] = dfl2['last_day_value']
        dfl2 = dfl2[['Stock Symbol','Bought at','Investment']]
    except:
        dfl2 = pd.DataFrame(columns = ['Stock Symbol','Bought at','Investment'])
    return dfl2

### Invest or Sell

In [6]:
def make_a_move(df3,dfl,path,num_investments = 2):
    if dfl.shape[0] == 0:
        balance = 1000
    else:
        balance = dfl['Investment'].sum()
    df3 = df3.iloc[:num_investments,]
    df3['Investment'] = balance/num_investments
    dflf = df3[['Stock Symbol','last_day_value','Investment']]
    dflf.columns = ['Stock Symbol','Bought at','Investment']
    dflf.to_csv(path)
    return balance

### Lets play

In [ ]:
df = read_stock_data(path = "main_data.csv")
l = [['Date','Balance']]
date = '2020-01-01'
num_investments = 10

ls = np.unique(df['Stock Symbol'])

while date < df['Date'].max():
    try:
        dfb = pd.read_csv('balances.csv')
        date = (pd.to_datetime(dfb['Date'].max(),format='%Y-%m-%d') + pd.DateOffset(1)).strftime('%Y-%m-%d')
    except:
        dfb = pd.DataFrame(columns = ['Date','Balance'])
        date = '2020-01-01'

    df3 = create_decision_data(df, date, ls)
    dfl2 = update_ledger(df3, path = 'ledger.csv')
    balance = make_a_move(df3,dfl2,num_investments = num_investments, path = 'ledger.csv')
    l.append([date,balance])
    
    df4 = pd.DataFrame(l)
    df4.columns = df4.iloc[0]
    df4 = df4.drop(df4.index[0])
    
    df4.to_csv('balances.csv')
    print(date,df['Date'].max())
    print('Next iteration :',date < df['Date'].max())

Performing stepwise search to minimize aic
Fit ARIMA: (2, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=2596.427, BIC=2630.753, Time=0.939 seconds
Fit ARIMA: (0, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=2593.019, BIC=2604.461, Time=0.137 seconds
Fit ARIMA: (1, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=2592.094, BIC=2609.256, Time=0.122 seconds
Fit ARIMA: (0, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=2591.962, BIC=2609.125, Time=0.166 seconds
Fit ARIMA: (0, 1, 0)x(0, 0, 0, 0) (constant=False); AIC=2592.479, BIC=2598.200, Time=0.078 seconds
Fit ARIMA: (1, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=2592.585, BIC=2615.469, Time=0.532 seconds
Fit ARIMA: (0, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=2592.915, BIC=2615.799, Time=0.294 seconds
Fit ARIMA: (1, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=2594.380, BIC=2622.985, Time=0.956 seconds
Total fit time: 3.229 seconds
Performing stepwise search to minimize aic
Fit ARIMA: (2, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=5981.819, BIC=6016.799, Time=2.363 seconds
F

Fit ARIMA: (1, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=4227.491, BIC=4250.573, Time=0.236 seconds
Fit ARIMA: (1, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=4225.936, BIC=4260.560, Time=1.608 seconds
Fit ARIMA: (3, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=4225.850, BIC=4260.474, Time=1.299 seconds
Fit ARIMA: (3, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=4221.791, BIC=4267.956, Time=3.422 seconds
Fit ARIMA: (4, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=4209.409, BIC=4261.345, Time=2.997 seconds
Fit ARIMA: (4, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=4207.873, BIC=4254.038, Time=2.922 seconds
Fit ARIMA: (4, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=4221.718, BIC=4262.113, Time=1.333 seconds
Fit ARIMA: (5, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=4209.215, BIC=4261.150, Time=4.665 seconds
Fit ARIMA: (5, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=4219.700, BIC=4265.866, Time=1.364 seconds
Fit ARIMA: (5, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=4211.341, BIC=4269.047, Time=3.067 seconds
Total fit time: 35.0

Fit ARIMA: (3, 1, 4)x(0, 0, 0, 0) (constant=True); AIC=6852.831, BIC=6905.305, Time=4.734 seconds
Fit ARIMA: (2, 1, 4)x(0, 0, 0, 0) (constant=True); AIC=6872.939, BIC=6919.582, Time=4.002 seconds
Fit ARIMA: (4, 1, 4)x(0, 0, 0, 0) (constant=True); AIC=6855.886, BIC=6914.190, Time=4.858 seconds
Fit ARIMA: (3, 1, 5)x(0, 0, 0, 0) (constant=True); AIC=6853.408, BIC=6911.712, Time=5.958 seconds
Fit ARIMA: (2, 1, 5)x(0, 0, 0, 0) (constant=True); AIC=6853.870, BIC=6906.344, Time=5.018 seconds
Fit ARIMA: (4, 1, 5)x(0, 0, 0, 0) (constant=True); AIC=6855.283, BIC=6919.418, Time=5.990 seconds
Total fit time: 52.115 seconds
Performing stepwise search to minimize aic
Fit ARIMA: (2, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=-5084.623, BIC=-5049.640, Time=1.778 seconds
Fit ARIMA: (0, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=-5089.443, BIC=-5077.783, Time=0.181 seconds
Fit ARIMA: (1, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=-5088.051, BIC=-5070.560, Time=0.256 seconds
Fit ARIMA: (0, 1, 1)x(0, 0, 0, 0) (con

Fit ARIMA: (1, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=5545.700, BIC=5569.021, Time=0.661 seconds
Fit ARIMA: (1, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=5547.348, BIC=5582.330, Time=2.061 seconds
Fit ARIMA: (3, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=5547.328, BIC=5582.311, Time=2.022 seconds
Fit ARIMA: (3, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=5539.685, BIC=5586.329, Time=4.204 seconds
Near non-invertible roots for order (3, 1, 3)(0, 0, 0, 0); setting score to inf (at least one inverse root too close to the border of the unit circle: 0.993)
Total fit time: 22.314 seconds
Performing stepwise search to minimize aic
Fit ARIMA: (2, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=14038.411, BIC=14073.394, Time=2.694 seconds
Fit ARIMA: (0, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=14045.542, BIC=14057.203, Time=0.050 seconds
Fit ARIMA: (1, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=14047.427, BIC=14064.918, Time=0.143 seconds
Fit ARIMA: (0, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=14047.434, BIC=14064.

Fit ARIMA: (3, 1, 5)x(0, 0, 0, 0) (constant=True); AIC=6855.147, BIC=6913.455, Time=5.830 seconds
Fit ARIMA: (2, 1, 5)x(0, 0, 0, 0) (constant=True); AIC=6855.594, BIC=6908.071, Time=4.387 seconds
Fit ARIMA: (4, 1, 5)x(0, 0, 0, 0) (constant=True); AIC=6857.023, BIC=6921.162, Time=6.175 seconds
Total fit time: 48.108 seconds
Performing stepwise search to minimize aic
Fit ARIMA: (2, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=-5085.745, BIC=-5050.760, Time=3.196 seconds
Fit ARIMA: (0, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=-5090.856, BIC=-5079.194, Time=0.211 seconds
Fit ARIMA: (1, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=-5089.546, BIC=-5072.053, Time=0.422 seconds
Fit ARIMA: (0, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=-5089.518, BIC=-5072.026, Time=0.238 seconds
Fit ARIMA: (0, 1, 0)x(0, 0, 0, 0) (constant=False); AIC=-5091.980, BIC=-5086.149, Time=0.078 seconds
Fit ARIMA: (1, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=-5086.637, BIC=-5063.314, Time=0.482 seconds
Total fit time: 4.634 seconds
P

...

Fit ARIMA: (2, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=5994.097, BIC=6034.924, Time=3.552 seconds
Fit ARIMA: (1, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=6001.589, BIC=6024.918, Time=0.221 seconds
Fit ARIMA: (1, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=6002.303, BIC=6037.297, Time=1.699 seconds
Fit ARIMA: (3, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=6002.141, BIC=6037.135, Time=0.897 seconds
Fit ARIMA: (3, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=5996.918, BIC=6043.578, Time=2.949 seconds
Total fit time: 18.606 seconds
Performing stepwise search to minimize aic
Fit ARIMA: (2, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=11693.858, BIC=11728.853, Time=1.236 seconds
Fit ARIMA: (0, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=11692.895, BIC=11704.559, Time=0.043 seconds
Fit ARIMA: (1, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=11692.616, BIC=11710.113, Time=0.148 seconds
Fit ARIMA: (0, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=11692.755, BIC=11710.252, Time=0.188 seconds
Fit ARIMA: (0, 1, 0)x(0, 0, 0, 0) (c

Fit ARIMA: (2, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=4046.029, BIC=4078.019, Time=2.099 seconds
Fit ARIMA: (0, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=4047.366, BIC=4058.029, Time=0.125 seconds
Fit ARIMA: (1, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=4049.334, BIC=4065.329, Time=0.090 seconds
Fit ARIMA: (0, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=4049.333, BIC=4065.329, Time=0.115 seconds
Fit ARIMA: (0, 1, 0)x(0, 0, 0, 0) (constant=False); AIC=4048.709, BIC=4054.041, Time=0.033 seconds
Fit ARIMA: (1, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=4051.416, BIC=4078.074, Time=1.178 seconds
Fit ARIMA: (2, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=4053.319, BIC=4079.977, Time=0.426 seconds
Fit ARIMA: (3, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=4051.838, BIC=4089.160, Time=2.292 seconds
Near non-invertible roots for order (3, 1, 2)(0, 0, 0, 0); setting score to inf (at least one inverse root too close to the border of the unit circle: 0.991)
Fit ARIMA: (2, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=40

Fit ARIMA: (1, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=2607.135, BIC=2630.031, Time=1.543 seconds
Fit ARIMA: (1, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=2610.637, BIC=2644.981, Time=0.704 seconds
Fit ARIMA: (3, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=2610.504, BIC=2644.848, Time=1.675 seconds
Fit ARIMA: (3, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=2603.454, BIC=2649.246, Time=2.771 seconds
Total fit time: 19.604 seconds
Performing stepwise search to minimize aic
Fit ARIMA: (2, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=6000.013, BIC=6035.010, Time=3.031 seconds
Fit ARIMA: (0, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=6003.753, BIC=6015.418, Time=0.214 seconds
Fit ARIMA: (1, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=6005.698, BIC=6023.196, Time=0.139 seconds
Fit ARIMA: (0, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=6005.697, BIC=6023.195, Time=0.208 seconds
Fit ARIMA: (0, 1, 0)x(0, 0, 0, 0) (constant=False); AIC=6003.438, BIC=6009.270, Time=0.036 seconds
Fit ARIMA: (1, 1, 2)x(0, 0, 0, 0) (constant

Fit ARIMA: (2, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=11347.014, BIC=11387.843, Time=1.669 seconds
Fit ARIMA: (1, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=11345.431, BIC=11368.763, Time=0.466 seconds
Fit ARIMA: (1, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=11348.236, BIC=11383.233, Time=0.586 seconds
Fit ARIMA: (3, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=11345.949, BIC=11380.946, Time=0.977 seconds
Fit ARIMA: (3, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=11347.200, BIC=11393.863, Time=3.546 seconds
Total fit time: 14.590 seconds
Performing stepwise search to minimize aic
Fit ARIMA: (2, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=8595.650, BIC=8630.647, Time=0.540 seconds
Fit ARIMA: (0, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=8587.652, BIC=8599.317, Time=0.046 seconds
Fit ARIMA: (1, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=8589.592, BIC=8607.090, Time=0.140 seconds
Fit ARIMA: (0, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=8589.592, BIC=8607.091, Time=0.160 seconds
Fit ARIMA: (0, 1, 0)x(0, 0, 0, 0) 

Fit ARIMA: (2, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=2855.158, BIC=2878.489, Time=0.227 seconds
Total fit time: 6.505 seconds
2020-01-10 2020-02-28
Next iteration : True
Performing stepwise search to minimize aic
Fit ARIMA: (2, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=2602.688, BIC=2637.032, Time=2.542 seconds
Fit ARIMA: (0, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=2608.166, BIC=2619.614, Time=0.204 seconds
Fit ARIMA: (1, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=2606.731, BIC=2623.903, Time=0.152 seconds
Fit ARIMA: (0, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=2606.562, BIC=2623.734, Time=0.264 seconds
Fit ARIMA: (0, 1, 0)x(0, 0, 0, 0) (constant=False); AIC=2607.576, BIC=2613.300, Time=0.110 seconds
Fit ARIMA: (1, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=2608.666, BIC=2637.286, Time=1.091 seconds
Fit ARIMA: (2, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=2608.586, BIC=2637.206, Time=0.776 seconds
Fit ARIMA: (3, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=2603.038, BIC=2643.106, Time=4.292 seconds


In [25]:
df = read_stock_data(path = "main_data.csv")

df = df.head()

df

,Date,Stock Symbol,Close
0,2010-01-04,CRK,216.600006
35323,2010-01-04,SKX,9.753333
19987,2010-01-04,TDG,49.400002
22543,2010-01-04,BBX,0.350000
32767,2010-01-04,ATV,32.599998


In [26]:
# df.append(pd.DataFrame({'Date':'2010-01-04','Close':'ATV','Stock Symbol':'32.599998'}))
df.append(pd.DataFrame(data = {'Date':['2010-01-04'],'Close':[32.599998],'Stock Symbol':['ATV']})).reset_index().drop(columns = 'index')

,Close,Date,Stock Symbol
0,216.600006,2010-01-04,CRK
1,9.753333,2010-01-04,SKX
2,49.400002,2010-01-04,TDG
3,0.350000,2010-01-04,BBX
4,32.599998,2010-01-04,ATV
5,32.599998,2010-01-04,ATV


In [ ]:
# p = d = q = range(0, 2)
# pdq = list(itertools.product(p, d, q))
# seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
# print('Examples of parameter combinations for Seasonal ARIMA...')
# print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
# print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
# print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
# print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

# y = df2t['Close'].resample('MS').mean()
# print(y['2017':])

# # y.plot(figsize=(15, 6))
# # plt.show()

# rcParams['figure.figsize'] = 18, 8
# decomposition = sm.tsa.seasonal_decompose(y, model='additive')
# fig = decomposition.plot()
# plt.show()

# for param in pdq:
#     for param_seasonal in seasonal_pdq:
#         try:
#             mod = sm.tsa.statespace.SARIMAX(y,
#                                             order=param,
#                                             seasonal_order=param_seasonal,
#                                             enforce_stationarity=False,
#                                             enforce_invertibility=False)
#             results = mod.fit()
#             print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
#         except:
#             continue

### Auto-Arima for forecast

In [ ]:
# model = auto_arima(df2t, trace=True, error_action='ignore', suppress_warnings=True)
# model.fit(df2t)

# next_day_forecast = model.predict(n_periods=1)[0]
# AIC = model.aic()
# next_day_value = df2v.iloc[:1,].values[0][0]
# last_day_value = df2t.iloc[-1:,].values[0][0]
# row = [i, last_day_value, next_day_forecast, AIC, next_day_value]

# forecast = model.predict(n_periods=len(df2v))
# forecast = pd.DataFrame(forecast,index = df2v.index,columns=['Prediction'])

# #plot the predictions for validation set
# plt.plot(df2t, label='Train', color = 'b')
# plt.plot(df2v, label='Valid', color = 'g')
# plt.plot(forecast, label='Prediction', color = 'r')
# plt.show()